# DBAAS Hello World and Test Truncate

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<repositories>
<repository>
   <id>snapshots_private</id>
   <url>file:/home/training/.m2/repository/</url>
</repository>
</repositories>

<dependencies>

  <dependency>
    <groupId>io.netty</groupId>
    <artifactId>netty-transport</artifactId>
     <version>4.1.87.Final</version>
  </dependency>

  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.1.9-SNAPSHOT</version>
  </dependency>
  
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-proxy-client</artifactId>
    <version>6.1.9-SNAPSHOT</version>
  </dependency>

</dependencies>

# Test Hello World!

In [3]:
import com.aerospike.client.proxy.AerospikeClientProxy;
import com.aerospike.client.policy.*;
import com.aerospike.client.*;
import com.aerospike.client.AerospikeException;
import com.aerospike.client.ResultCode;

//Required Imports
//import com.aerospike.client.AerospikeClient;
//import com.aerospike.client.policy.WritePolicy;
//import com.aerospike.client.Bin;
//import com.aerospike.client.Key;
//import com.aerospike.client.Record;
//import com.aerospike.client.Value;
//import com.aerospike.client.Operation;
import com.aerospike.client.query.Statement;
import com.aerospike.client.exp.Exp;

final String AEROSPIKE_CLOUD_HOST="f88a621d-69bf-4ba4-a825-0a17568628b2.asdb.io";
final String AEROSPIKE_CLOUD_KEY="fd1520c45b034a06a4c52e3d042aa022"; 
final String AEROSPIKE_CLOUD_SECRET="5851a650f3784756ae0f36ee5933df57";
final String AEROSPIKE_CLOUD_NAMESPACE="aerospike_cloud";
final int AEROSPIKE_CLOUD_PORT=4000;

ClientPolicy policy = new ClientPolicy();
policy.user = AEROSPIKE_CLOUD_KEY;
policy.password = AEROSPIKE_CLOUD_SECRET;

TlsPolicy tlsPolicy = new TlsPolicy();
policy.tlsPolicy = tlsPolicy;

AerospikeClientProxy client = new AerospikeClientProxy(policy, 
           new Host(AEROSPIKE_CLOUD_HOST, AEROSPIKE_CLOUD_PORT) 
                );

Key key = new Key(AEROSPIKE_CLOUD_NAMESPACE, "my_set", "my_key");
Bin bin = new Bin("my_bin", "Hello World!");

client.put(null, key, bin);
Record record = client.get(null, key);
System.out.println(record);

//client.close();


(gen:1),(exp:0),(bins:(my_bin:Hello World!))


# Test Truncate ... 

In [4]:
//Insert a record using a class based construct
class userProfile {
  public void createProfile (String ns, String setName, int profileId, String name, int zip, boolean bCheck) {
    Key user = new Key(ns, setName, profileId); 
    Bin bProfileId = new Bin("ProfileId", profileId);
    Bin bName = new Bin("Name", name);
    Bin bZip = new Bin("Zip", zip);
    WritePolicy wPolicy1 = new WritePolicy();
    client.put(wPolicy1, user, bProfileId, bName, bZip);
    if(bCheck){
      System.out.println(client.get(null, user));
    }   
  }
}

In [5]:
userProfile pf = new userProfile();
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", 0,"Test", 94506, true);

(gen:1),(exp:0),(bins:(ProfileId:0),(Name:Test),(Zip:94506))


In [8]:
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset", 4,"Test", 94506, false);

In [9]:
Key user4 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset", 4); 
System.out.println(client.get(null, user4));

(gen:1),(exp:0),(bins:(ProfileId:4),(Name:Test),(Zip:94506))


In [12]:
class Truncate {
  public void truncate(String ns, String setName, int sinceUpdate_ms) {
    WritePolicy wPolicy = new WritePolicy();
    if(sinceUpdate_ms > 0){
      //Add "Since Last Update Time" based Expression Filter
      wPolicy.filterExp = Exp.build(Exp.gt(Exp.sinceUpdate(), Exp.val(sinceUpdate_ms)));
    }
    
    Operation[] ops = new Operation[] {
      Operation.delete()
    };
    Statement stmt = new Statement();
    stmt.setNamespace(ns);
    if(setName != null){
      stmt.setSetName(setName);
    }
    stmt.setOperations(ops);
    client.execute(wPolicy, stmt);
  }
}

In [13]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, "testset", 0);

In [14]:
System.out.println(client.get(null, user4));

null


In [15]:
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset1", 1,"Test", 94506, false);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset2", 2,"Test", 94506, false);
pf.createProfile(AEROSPIKE_CLOUD_NAMESPACE, "testset3", 3,"Test", 94506, false);


In [16]:
Key user1 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset1", 1); 
Key user2 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset2", 2);
Key user3 = new Key(AEROSPIKE_CLOUD_NAMESPACE, "testset3", 3); 

System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));

(gen:1),(exp:0),(bins:(ProfileId:1),(Name:Test),(Zip:94506))
(gen:1),(exp:0),(bins:(ProfileId:2),(Name:Test),(Zip:94506))
(gen:1),(exp:0),(bins:(ProfileId:3),(Name:Test),(Zip:94506))


In [17]:
Truncate tr = new Truncate();
tr.truncate(AEROSPIKE_CLOUD_NAMESPACE, null, 10000);

In [18]:
System.out.println(client.get(null, user1));
System.out.println(client.get(null, user2));
System.out.println(client.get(null, user3));

null
null
null


In [19]:
client.close();